### Show some of the largest fires, modis cmp viirs

In [ ]:
import h5py
from matplotlib import animation
from wildfire.data_utils import *
from wildfire.data_types import *
from wildfire.dataset_utils import *
import matplotlib.pyplot as plt
from einops import reduce
from mpl_toolkits.axes_grid1 import make_axes_locatable
is_test = False
viirs = h5_get_path("viirs", is_test)
print(viirs)
viirs = h5py.File(viirs, "r")
modis = h5_get_path("modis", is_test)
modis = h5py.File(modis, "r")
dates = list(viirs["num_fire_pixels_by_day"].keys())

fires_path = os.path.join(config.root_path, "fires", "fires.json")
fires = json_load(fires_path)

def one_fire(ax, fire, ds, patch_size_org):
    patch_size = config.patch_size_km
    gx0, gy0, gx1, gy1 = fire["grid_bounds"]
    # Divide by 3 to get km resolution
    px0, py0, px1, py1 = [x // 3 for x in fire["pixel_bounds"]]
    xs = (px1 - px0) + 1
    ys = (py1 - py0) + 1
    start_date = fire["start_date"]
    end_date = fire["end_date"]
    t0 = dates.index(start_date)
    t1 = dates.index(end_date)
    default_val = np.full((patch_size_org, patch_size_org), 0, dtype=np.int16)
    def get_day(t, y, x, day_type):
        xy_path = H5Grid.get_cell_path(x, y)
        day = h5_get_nested(ds, ["cells", xy_path, dates[t], day_type, "fire_mask"])
        if day is None:
            return default_val
        return day[...][0]
    num_fires = 0
    T = t1 - t0 + 1
    img = np.full((T, ys, xs), -1, dtype=np.int16)
    for x in range(gx0, gx1 + 1):
        for y in range(gy0, gy1 + 1):
            for t in range(t0, t1 + 1):
                ti = t - t0
                day = get_day(t, y, x, "day")
                night = get_day(t, y, x, "night")
                num_fires += np.sum(day > 6) + np.sum(night > 6)
                new = np.maximum(day, night)
                if patch_size != new.shape[-1]:
                    new = reduce(new, "(h 3) (w 3) -> h w", "max", h=patch_size, w=patch_size)
                xo = (x - gx0) * patch_size - px0
                yo = (gy1 - y) * patch_size - py0
                fill = np.full((patch_size, patch_size), -1, dtype=np.int16)
                fill[new > 6] = (t - t0)
                x0 = max(0, xo)
                y0 = max(0, yo)
                x1 = min(xs - 1, xo + patch_size)
                y1 = min(ys - 1, yo + patch_size)
                if x1 - x0 <= 0 or y1 - y0 <= 0:
                    continue
                old = img[ti, y0:y1, x0:x1]
                fill = fill[y0 - yo:y1 - yo, x0 - xo:x1 - xo]
                res = np.maximum(old, fill)
                img[ti, y0:y1, x0:x1] = res

    for t in range(1, T):
        img[t] = np.maximum(img[t], img[t - 1])
    # img = -img
    # Create a masked array where 0 values are masked
    masked_img = np.ma.masked_where(img == -1, img)
    return masked_img


i0 = 0
idxs = [0, 2, 6, 25, 27, 33]
idxs2 = [25, 27, 29, 33]
dir = os.path.join(config.root_path, "figures", "fires_train")
os.makedirs(dir, exist_ok=True)

for i in range(i0, i0 + 35):
# for i in idxs:
    print(i)
    
    im1 = one_fire(None, fires[i], viirs, config.patch_size)
    im2 = one_fire(None, fires[i], modis, config.patch_size_km)
    
    # Create figure and animation
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    
    background = np.ones((*im1.shape[1:], 3)) * 0.85  # Light gray background
    vmax = max(im1.max(), im2.max())
    
    ax1.set_title("VNP14", fontsize=22)
    ax1.imshow(background)
    ax1.axis("off")
    im1_plot = ax1.imshow(np.zeros_like(im1[0]), cmap="YlOrRd", vmin=0, vmax=vmax)
    
    ax2.set_title("MOD14", fontsize=22) 
    ax2.imshow(background)
    ax2.axis("off")
    im2_plot = ax2.imshow(np.zeros_like(im2[0]), cmap="YlOrRd", vmin=0, vmax=vmax)
    
    divider = make_axes_locatable(ax2)
    cax = divider.append_axes("right", size="5%", pad=0.15)
    cbar = plt.colorbar(im2_plot, cax=cax)
    cbar.set_label("Days since fire start", size=16)
    cbar.ax.yaxis.set_tick_params(labelsize=16)

    def animate(frame):
        im1_plot.set_array(im1[frame].astype(np.float32))
        ax1.set_title(f"VNP14: t={frame}")
        im2_plot.set_array(im2[frame].astype(np.float32))
        ax2.set_title(f"MOD14: t={frame}")
        return [im1_plot, im2_plot]

    anim = animation.FuncAnimation(fig, animate, frames=len(im1), interval=200)
    gif = f"{dir}/fire_{i}.gif"
    anim.save(gif, writer='pillow')
    print(gif)
    plt.close()




In [ ]:
# ======== Task at hand ========


import h5py
from matplotlib import animation
from wildfire.data_utils import *
from wildfire.data_types import *
from wildfire.dataset_utils import *
import matplotlib.pyplot as plt
from einops import reduce
from mpl_toolkits.axes_grid1 import make_axes_locatable
is_test = False
viirs = h5_get_path("viirs", is_test)
print(viirs)
viirs = h5py.File(viirs, "r")
modis = h5_get_path("modis", is_test)
modis = h5py.File(modis, "r")
dates = list(viirs["num_fire_pixels_by_day"].keys())

fires_path = os.path.join(config.root_path, "fires", "fires.json")
fires = json_load(fires_path)

def one_fire(ax, fire, ds, patch_size_org):
    patch_size = config.patch_size_km
    gx0, gy0, gx1, gy1 = fire["grid_bounds"]
    # Divide by 3 to get km resolution
    px0, py0, px1, py1 = [x // 3 for x in fire["pixel_bounds"]]
    xs = (px1 - px0) + 1
    ys = (py1 - py0) + 1
    start_date = fire["start_date"]
    end_date = fire["end_date"]
    t0 = dates.index(start_date)
    t1 = dates.index(end_date)
    default_val = np.full((patch_size_org, patch_size_org), 0, dtype=np.int16)
    def get_day(t, y, x, day_type):
        xy_path = H5Grid.get_cell_path(x, y)
        day = h5_get_nested(ds, ["cells", xy_path, dates[t], day_type, "fire_mask"])
        if day is None:
            return default_val
        return day[...][0]
    num_fires = 0
    T = t1 - t0 + 1
    img = np.full((T, ys, xs), -1, dtype=np.int16)
    for x in range(gx0, gx1 + 1):
        for y in range(gy0, gy1 + 1):
            for t in range(t0, t1 + 1):
                ti = t - t0
                day = get_day(t, y, x, "day")
                night = get_day(t, y, x, "night")
                num_fires += np.sum(day > 6) + np.sum(night > 6)
                new = np.maximum(day, night)
                if patch_size != new.shape[-1]:
                    new = reduce(new, "(h 3) (w 3) -> h w", "max", h=patch_size, w=patch_size)
                xo = (x - gx0) * patch_size - px0
                yo = (gy1 - y) * patch_size - py0
                fill = np.full((patch_size, patch_size), -1, dtype=np.int16)
                fill[new > 6] = (t - t0)
                x0 = max(0, xo)
                y0 = max(0, yo)
                x1 = min(xs - 1, xo + patch_size)
                y1 = min(ys - 1, yo + patch_size)
                if x1 - x0 <= 0 or y1 - y0 <= 0:
                    continue
                old = img[ti, y0:y1, x0:x1]
                fill = fill[y0 - yo:y1 - yo, x0 - xo:x1 - xo]
                res = np.maximum(old, fill)
                img[ti, y0:y1, x0:x1] = res

    for t in range(1, T):
        img[t] = np.maximum(img[t], img[t - 1])
    # img = -img
    # Create a masked array where 0 values are masked
    masked_img = np.ma.masked_where(img == -1, img)
    return masked_img


i0 = 0
idxs = [25]
dir = os.path.join(config.root_path, "figures", "fires_train")
os.makedirs(dir, exist_ok=True)

# for i in range(i0, i0 + 35):
for i in idxs:
    print(i)
    
    im1 = one_fire(None, fires[i], viirs, config.patch_size)
    
    # Create figure and animation
    fig, ax1 = plt.subplots(figsize=(10, 5))
    
    background = np.ones((*im1.shape[1:], 3)) * 0.85  # Light gray background
    vmax = im1.max()
    
    ax1.set_title("Fire", fontsize=22)
    ax1.imshow(background)
    ax1.axis("off")
    im1_plot = ax1.imshow(np.zeros_like(im1[0]), cmap="YlOrRd", vmin=0, vmax=vmax)
    divider = make_axes_locatable(ax1)
    cax = divider.append_axes("right", size="5%", pad=0.15)
    cbar = plt.colorbar(im1_plot, cax=cax)
    cbar.set_label("Days since fire start", size=16)
    cbar.ax.yaxis.set_tick_params(labelsize=16)

    def animate(frame):
        im1_plot.set_array(im1[frame].astype(np.float32))
        ax1.set_title(f"Fire at $t={frame}$")
        return [im1_plot]

    anim = animation.FuncAnimation(fig, animate, frames=len(im1), interval=500)
    gif = f"{dir}/task_fire_{i}.gif"
    anim.save(gif, writer='pillow')
    print(gif)
    plt.close()


